In [1]:
import os
import pandas as pd
import numpy as np
import cv2
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from tensorflow.keras.preprocessing.image import img_to_array, load_img
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.utils.class_weight import compute_class_weight

# Path to dataset
dataset_path = r"D:\ATTANDANCE\DataSet Sept-Dec2023\data attendence 031023\Photoes"

# Collect image paths and labels
image_paths = []
labels = []

for roll_number in os.listdir(dataset_path):
    roll_folder = os.path.join(dataset_path, roll_number)
    
    if os.path.isdir(roll_folder):
        for img_name in os.listdir(roll_folder):
            img_path = os.path.join(roll_folder, img_name)
            # Check if file is an image
            if img_name.lower().endswith(('.png', '.jpg', '.jpeg')):
                image_paths.append(img_path)
                labels.append(roll_number)

# Create DataFrame
df = pd.DataFrame({
    'image_path': image_paths,
    'label': labels
})

# Load images and preprocess
def load_and_preprocess_image(img_path, target_size=(100, 100)):
    image = load_img(img_path, target_size=target_size)
    image = img_to_array(image)
    image = image / 255.0  # Normalize
    return image

# Prepare data for training
X = []
y = []

for _, row in df.iterrows():
    img = load_and_preprocess_image(row['image_path'])
    X.append(img)
    y.append(row['label'])

X = np.array(X)
y = np.array(y)

# Encode labels
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)
y_encoded = to_categorical(y_encoded)  # One-hot encode labels

# Split data
X_train, X_test, y_train, y_test = train_test_split(X, y_encoded, test_size=0.2, random_state=42)

# Compute class weights
class_weights = compute_class_weight('balanced', classes=np.unique(np.argmax(y_encoded, axis=1)), y=np.argmax(y_encoded, axis=1))
class_weights = dict(enumerate(class_weights))

# Build CNN model
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(100, 100, 3)),
    MaxPooling2D(pool_size=(2, 2)),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D(pool_size=(2, 2)),
    Flatten(),
    Dense(128, activation='relu'),
    Dense(len(label_encoder.classes_), activation='softmax')
])

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Train model
history = model.fit(X_train, y_train, epochs=10, batch_size=32, validation_split=0.1, class_weight=class_weights)

# Evaluate model
test_loss, test_accuracy = model.evaluate(X_test, y_test)
print(f'Test Loss: {test_loss}')
print(f'Test Accuracy: {test_accuracy}')

# Save the model and label encoder classes
model.save(r'D:\ATTANDANCE\face_recognition_model.h5')
np.save(r'D:\ATTANDANCE\classes.npy', label_encoder.classes_)


C:\Users\DAXESH MAHERIYA\anaconda3\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:99: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(


Epoch 1/10
11/11 ━━━━━━━━━━━━━━━━━━━━ 6s 148ms/step - accuracy: 0.0207 - loss: 4.2154 - val_accuracy: 0.0000e+00 - val_loss: 3.7008
Epoch 2/10
11/11 ━━━━━━━━━━━━━━━━━━━━ 1s 93ms/step - accuracy: 0.0611 - loss: 3.6576 - val_accuracy: 0.0000e+00 - val_loss: 3.6775
Epoch 3/10
11/11 ━━━━━━━━━━━━━━━━━━━━ 1s 94ms/step - accuracy: 0.0761 - loss: 3.5645 - val_accuracy: 0.0526 - val_loss: 3.5791
Epoch 4/10
11/11 ━━━━━━━━━━━━━━━━━━━━ 1s 96ms/step - accuracy: 0.2385 - loss: 3.2059 - val_accuracy: 0.1579 - val_loss: 3.3595
Epoch 5/10
11/11 ━━━━━━━━━━━━━━━━━━━━ 1s 97ms/step - accuracy: 0.4191 - loss: 2.5425 - val_accuracy: 0.2895 - val_loss: 3.1063
Epoch 6/10
11/11 ━━━━━━━━━━━━━━━━━━━━ 1s 94ms/step - accuracy: 0.6368 - loss: 1.8530 - val_accuracy: 0.3158 - val_loss: 2.5606
Epoch 7/10
11/11 ━━━━━━━━━━━━━━━━━━━━ 1s 99ms/step - accuracy: 0.7039 - loss: 1.2388 - val_accuracy: 0.2895 - val_loss: 2.9737
Epoch 8/10
11/11 ━━━━━━━━━━━━━━━━━━━━ 1s 99ms/step - accuracy: 0.8536 - loss: 0.6758 - val_accuracy: 0

Test Loss: 3.5038554668426514
Test Accuracy: 0.3978494703769684


In [2]:
import cv2
import numpy as np
import pandas as pd
import os
from tensorflow.keras.models import load_model
from keras.preprocessing.image import img_to_array

# Paths
model_path = r'D:\ATTANDANCE\face_recognition_model.h5'
classes_path = r'D:\ATTANDANCE\classes.npy'
video_folder_path = r"D:\ATTANDANCE\DataSet Sept-Dec2023\CCTV Footage 05_10_2023"
output_folder_path = r"C:\Users\DAXESH MAHERIYA\OneDrive\Desktop\TEST"
enhanced_images_folder = r"C:\Users\DAXESH MAHERIYA\OneDrive\Desktop\img"

# Load the trained model
model = load_model(model_path)
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Load the label encoder classes
label_encoder = LabelEncoder()
label_encoder.classes_ = np.load(classes_path)

# Ensure output folders exist
os.makedirs(output_folder_path, exist_ok=True)
os.makedirs(enhanced_images_folder, exist_ok=True)

# Function to extract and enhance frames from a video file
def extract_and_enhance_frames(video_path, num_frames=30):
    cap = cv2.VideoCapture(video_path)
    frames = []
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    frame_step = max(total_frames // num_frames, 1)
    
    for i in range(num_frames):
        cap.set(cv2.CAP_PROP_POS_FRAMES, i * frame_step)
        ret, frame = cap.read()
        if ret:
            # Enhance the image
            enhanced_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
            enhanced_frame = cv2.equalizeHist(enhanced_frame)
            enhanced_frame = cv2.cvtColor(enhanced_frame, cv2.COLOR_GRAY2BGR)
            
            # Save enhanced image
            img_filename = f"{os.path.splitext(os.path.basename(video_path))[0]}_frame_{i+1}.jpg"
            img_path = os.path.join(enhanced_images_folder, img_filename)
            cv2.imwrite(img_path, enhanced_frame)
            
            frames.append(enhanced_frame)
    cap.release()
    return frames

# Function to preprocess image and predict
def predict_frame(model, frame):
    image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    image = cv2.resize(image, (100, 100))
    image = img_to_array(image)
    image = np.expand_dims(image, axis=0)
    predictions = model.predict(image)
    return np.argmax(predictions[0])

# Process each video file
for video_file in os.listdir(video_folder_path):
    video_path = os.path.join(video_folder_path, video_file)
    
    if not video_file.lower().endswith('.dav'):
        continue
    
    try:
        frames = extract_and_enhance_frames(video_path)
        predictions = [predict_frame(model, frame) for frame in frames]
        
        # Create a DataFrame to hold roll number status
        roll_numbers = list(set(predictions))  # Get unique roll numbers detected
        attendance_records = []
        
        for roll_number in roll_numbers:
            # Mark as 'Present' if the roll number is detected in any frame
            status = 'Present' if roll_number in predictions else 'Absent'
            record = {'Roll Number': roll_number, 'Status': status}
            attendance_records.append(record)
        
        # Save the attendance records to an Excel file
        df = pd.DataFrame(attendance_records)
        excel_path = os.path.join(output_folder_path, f'{os.path.splitext(video_file)[0]}_attendance.xlsx')
        df.to_excel(excel_path, index=False)
        print(f"Attendance report saved to: {excel_path}")
        
    except Exception as e:
        print(f"Error processing video {video_file}: {e}") 


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 267ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 79ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 63ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 63ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 60ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 63ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 79ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 74ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 62ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 70ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 60ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 63ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 63ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 63ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 63ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 45ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 63ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 65ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 63ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 63ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 64ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 64ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 63ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 63ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 78ms/step
1/1 ━━━━━━━


KeyboardInterrupt



In [3]:
import os
import cv2

# Paths
video_folder_path = r"D:\ATTANDANCE\DataSet Sept-Dec2023\CCTV Footage 05_10_2023"
img_save_path = r"C:\Users\DAXESH MAHERIYA\OneDrive\Desktop\img"
output_folder_path = r"C:\Users\DAXESH MAHERIYA\OneDrive\Desktop\TEST"

# Ensure directories exist
os.makedirs(img_save_path, exist_ok=True)
os.makedirs(output_folder_path, exist_ok=True)

# Function to extract and save faces from frames
def extract_and_save_faces(video_path, num_frames=30):
    cap = cv2.VideoCapture(video_path)
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    frame_step = max(total_frames // num_frames, 1)
    face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

    for i in range(num_frames):
        cap.set(cv2.CAP_PROP_POS_FRAMES, i * frame_step)
        ret, frame = cap.read()
        if ret:
            gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
            faces = face_cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=5, minSize=(30, 30))

            for (x, y, w, h) in faces:
                face = frame[y:y + h, x:x + w]
                face_path = os.path.join(img_save_path, f'{os.path.splitext(os.path.basename(video_path))[0]}_frame{i}_face.jpg')
                cv2.imwrite(face_path, face)
    cap.release()

# Process each video file
for video_file in os.listdir(video_folder_path):
    video_path = os.path.join(video_folder_path, video_file)
    
    if not video_file.lower().endswith('.dav'):
        continue
    
    try:
        extract_and_save_faces(video_path)
        print(f"Faces extracted and saved from video: {video_file}")
        
    except Exception as e:
        print(f"Error processing video {video_file}: {e}")


Faces extracted and saved from video: 172.16.116.247_Class_1_1_main_20230925135557_161614.dav
Faces extracted and saved from video: 172.16.116.247_Class_1_1_main_20230926053403_063713.dav
Faces extracted and saved from video: 172.16.116.247_Class_1_1_main_20230926053749_063628.dav
Faces extracted and saved from video: 172.16.116.247_Class_1_1_main_20230926083030_095910.dav
Faces extracted and saved from video: 172.16.116.247_Class_1_1_main_20230927045628_053502.dav
Faces extracted and saved from video: 172.16.116.247_Class_1_1_main_20230929050421_052835.dav
Faces extracted and saved from video: 172.16.116.247_Class_1_1_main_20230929090204_111747.dav
Faces extracted and saved from video: 172.16.116.247_Class_3_1_main_20230929063149_072415.dav


In [4]:
from keras.preprocessing.image import img_to_array
from tensorflow.keras.models import load_model
import numpy as np
import pandas as pd

# Load the model and label encoder classes
model_path = r'D:\ATTANDANCE\face_recognition_model.h5'
classes_path = r'D:\ATTANDANCE\classes.npy'

try:
    model = load_model(model_path)
    label_encoder = LabelEncoder()
    label_encoder.classes_ = np.load(classes_path)
except Exception as e:
    print(f"Error loading model or classes: {e}")
    exit()

# Function to preprocess image and predict
def predict_face_image(image_path):
    image = cv2.imread(image_path)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    image = cv2.resize(image, (100, 100))  # Adjust size if necessary
    image = img_to_array(image)
    image = np.expand_dims(image, axis=0)
    predictions = model.predict(image)
    return np.argmax(predictions[0])

# Process saved face images for attendance
attendance_records = []

for face_file in os.listdir(img_save_path):
    face_path = os.path.join(img_save_path, face_file)
    
    try:
        predicted_label = predict_face_image(face_path)
        roll_number = label_encoder.classes_[predicted_label]
        attendance_records.append({'Roll Number': roll_number, 'Status': 'Present'})
        
    except Exception as e:
        print(f"Error processing face image {face_file}: {e}")

# Convert to DataFrame and save to Excel
df = pd.DataFrame(attendance_records).drop_duplicates()
excel_path = os.path.join(output_folder_path, 'attendance.xlsx')
df.to_excel(excel_path, index=False)
print(f"Attendance report saved to: {excel_path}")


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 95ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
1/1 ━━━━━━━━

In [1]:
import os
import pandas as pd
import numpy as np
import cv2
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from tensorflow.keras.preprocessing.image import img_to_array, load_img
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.utils.class_weight import compute_class_weight

# Path to dataset
dataset_path = r"D:\ATTANDANCE\DataSet Sept-Dec2023\data attendence 031023\Photoes"

# Collect image paths and labels
image_paths = []
labels = []

for roll_number in os.listdir(dataset_path):
    roll_folder = os.path.join(dataset_path, roll_number)
    
    if os.path.isdir(roll_folder):
        for img_name in os.listdir(roll_folder):
            img_path = os.path.join(roll_folder, img_name)
            # Check if file is an image
            if img_name.lower().endswith(('.png', '.jpg', '.jpeg')):
                image_paths.append(img_path)
                labels.append(roll_number)

# Create DataFrame
df = pd.DataFrame({
    'image_path': image_paths,
    'label': labels
})

# Load images and preprocess
def load_and_preprocess_image(img_path, target_size=(100, 100)):
    image = load_img(img_path, target_size=target_size)
    image = img_to_array(image)
    image = image / 255.0  # Normalize
    return image

# Prepare data for training
X = []
y = []

for _, row in df.iterrows():
    img = load_and_preprocess_image(row['image_path'])
    X.append(img)
    y.append(row['label'])

X = np.array(X)
y = np.array(y)

# Encode labels
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)
y_encoded = to_categorical(y_encoded)  # One-hot encode labels

# Split data
X_train, X_test, y_train, y_test = train_test_split(X, y_encoded, test_size=0.2, random_state=42)

# Compute class weights
class_weights = compute_class_weight('balanced', classes=np.unique(np.argmax(y_encoded, axis=1)), y=np.argmax(y_encoded, axis=1))
class_weights = dict(enumerate(class_weights))

# Build CNN model
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(100, 100, 3)),
    MaxPooling2D(pool_size=(2, 2)),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D(pool_size=(2, 2)),
    Flatten(),
    Dense(128, activation='relu'),
    Dense(len(label_encoder.classes_), activation='softmax')
])

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Train model
history = model.fit(X_train, y_train, epochs=10, batch_size=32, validation_split=0.1, class_weight=class_weights)

# Evaluate model
test_loss, test_accuracy = model.evaluate(X_test, y_test)
print(f'Test Loss: {test_loss}')
print(f'Test Accuracy: {test_accuracy}')

# Save the model and label encoder classes
model.save(r'D:\ATTANDANCE\face_recognition_model.h5')
np.save(r'D:\ATTANDANCE\classes.npy', label_encoder.classes_)

Epoch 1/10


C:\Users\DAXESH MAHERIYA\anaconda3\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:99: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(


11/11 ━━━━━━━━━━━━━━━━━━━━ 2s 64ms/step - accuracy: 0.0114 - loss: 4.3819 - val_accuracy: 0.0000e+00 - val_loss: 3.7574
Epoch 2/10
11/11 ━━━━━━━━━━━━━━━━━━━━ 1s 53ms/step - accuracy: 0.0426 - loss: 3.6998 - val_accuracy: 0.0000e+00 - val_loss: 3.7094
Epoch 3/10
11/11 ━━━━━━━━━━━━━━━━━━━━ 1s 52ms/step - accuracy: 0.0568 - loss: 3.6416 - val_accuracy: 0.0263 - val_loss: 3.6535
Epoch 4/10
11/11 ━━━━━━━━━━━━━━━━━━━━ 1s 52ms/step - accuracy: 0.0815 - loss: 3.5306 - val_accuracy: 0.0789 - val_loss: 3.5892
Epoch 5/10
11/11 ━━━━━━━━━━━━━━━━━━━━ 1s 51ms/step - accuracy: 0.2076 - loss: 3.2555 - val_accuracy: 0.0789 - val_loss: 3.5635
Epoch 6/10
11/11 ━━━━━━━━━━━━━━━━━━━━ 1s 50ms/step - accuracy: 0.2725 - loss: 2.8984 - val_accuracy: 0.2105 - val_loss: 3.2846
Epoch 7/10
11/11 ━━━━━━━━━━━━━━━━━━━━ 1s 50ms/step - accuracy: 0.5222 - loss: 2.2540 - val_accuracy: 0.1842 - val_loss: 3.2535
Epoch 8/10
11/11 ━━━━━━━━━━━━━━━━━━━━ 1s 51ms/step - accuracy: 0.6722 - loss: 1.4763 - val_accuracy: 0.2368 - val_

Test Loss: 3.394482374191284
Test Accuracy: 0.40860214829444885


In [3]:
import cv2
import numpy as np
import pandas as pd
import os
from tensorflow.keras.models import load_model
from keras.preprocessing.image import img_to_array

# Update these paths to the actual locations
model_path = r'D:\ATTANDANCE\face_recognition_model.h5'  # Ensure this path is correct
classes_path = r'D:\ATTANDANCE\classes.npy'  # Ensure this path is correct
video_folder_path = r"D:\ATTANDANCE\DataSet Sept-Dec2023\CCTV Footage 05_10_2023"
output_folder_path = r"C:\Users\DAXESH MAHERIYA\OneDrive\Desktop\TEST"

# Load the trained model
try:
    model = load_model(model_path)
    print("Model loaded successfully.")
    
    # Recompile the model if needed
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    print("Model recompiled successfully.")
except Exception as e:
    print(f"Error loading or recompiling model: {e}")
    exit()

# Load the label encoder classes
try:
    label_encoder = LabelEncoder()
    label_encoder.classes_ = np.load(classes_path)
    print("Label encoder classes loaded successfully.")
except Exception as e:
    print(f"Error loading label encoder classes: {e}")
    exit()

# Ensure output folder exists
os.makedirs(output_folder_path, exist_ok=True)

# Function to extract frames from a video file
def extract_frames(video_path, num_frames=10):
    cap = cv2.VideoCapture(video_path)
    frames = []
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    frame_step = max(total_frames // num_frames, 1)
    
    for i in range(num_frames):
        cap.set(cv2.CAP_PROP_POS_FRAMES, i * frame_step)
        ret, frame = cap.read()
        if ret:
            frames.append(frame)
    cap.release()
    return frames

# Function to preprocess image and predict
def predict_frame(model, frame):
    image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    image = cv2.resize(image, (100, 100))  # Adjust size if necessary
    image = img_to_array(image)
    image = np.expand_dims(image, axis=0)
    predictions = model.predict(image)
    return np.argmax(predictions[0])

# Process each video file
for video_file in os.listdir(video_folder_path):
    video_path = os.path.join(video_folder_path, video_file)
    
    if not video_file.lower().endswith('.dav'):
        continue
    
    try:
        frames = extract_frames(video_path)
        predictions = [predict_frame(model, frame) for frame in frames]
        
        # Create a DataFrame to hold roll number status
        roll_numbers = list(set(predictions))  # Get unique roll numbers detected
        attendance_records = []
        
        for roll_number in roll_numbers:
            # Determine attendance based on the detection in any frame
            status = 'Present' if roll_number in predictions else 'Absent'
            record = {'Roll Number': roll_number, 'Status': status}
            attendance_records.append(record)
        
        # Save the attendance records to an Excel file
        df = pd.DataFrame(attendance_records)
        excel_path = os.path.join(output_folder_path, f'{os.path.splitext(video_file)[0]}_attendance.xlsx')
        df.to_excel(excel_path, index=False)
        print(f"Attendance report saved to: {excel_path}")
        
    except Exception as e:
        print(f"Error processing video {video_file}: {e}")

Model loaded successfully.
Model recompiled successfully.
Label encoder classes loaded successfully.
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step
Attendance report saved to: C:\Users\DAXESH MAHERIYA\OneDrive\Desktop\TEST\172.16.116.247_Class_1_1_main_20230925135557_161614_attendance.xlsx
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
A